# Setting

## Import Library

In [334]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Date
from calendar import monthrange
from datetime import date, datetime

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

## Set Date Functions
모든 데이터에 대한 주차별 데이터를 사용할 예정이므로  
날짜 정보를 주차 컬럼으로 변경하는 함수 정의

In [335]:
from datetime import date, timedelta


def set_week(df, date):
    '''
    df : datetime 형식의 컬럼을 가지고 있는 dataframe
    date : df에서 datetime 형식을 가진 컬럼명
    return : date의 연도 컬럼과 주차 컬럼을 추가한 dataframe
    '''
    df[date] = pd.to_datetime(df[date])
    df[date] = df[date].dt.date
    df['year'] = df.apply(func=lambda x: x[date].isocalendar()[0], axis=1)
    df['week'] = df.apply(func=lambda x: x[date].isocalendar()[1], axis=1)
    df.drop(date, axis=1, inplace=True)
    

def check_week(df):
    '''
    df에 date가 전부 있는지 확인
    '''
    cnt = 0
    sdate = date(2015, 12, 28)   # start date
    edate = date(2019, 12, 30)   # end date
    delta = edate - sdate       # as timedelta
    mem = set()
    
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        year, week = day.isocalendar()[0], day.isocalendar()[1]
        if year * 100 + week in mem:
            continue
        mem.add(year * 100 + week)
        if df[(df['year'] == year) & (df['week'] == week)].empty:
            print((year, week), end="")
            cnt += 1
    if cnt > 0:
        print()
    print("missing", cnt, "values")    

## Set Data Root Directory

In [336]:
root = os.path.join(os.getcwd(), 'DATA')

# Oil

## Import Data

In [357]:
df_oil = pd.read_csv(os.path.join(root, 'raw_oil.csv'), usecols=["날짜", "종가"])
df_oil_dubai = pd.read_csv(os.path.join(root, 'raw_oil_dubai.csv'), usecols=["날짜", "종가"])
df_oil_brent = pd.read_csv(os.path.join(root, 'raw_oil_brent.csv'), usecols=["날짜", "종가"])

## Preprocess

In [358]:
from datetime import datetime
def preprocess_oil(df):
    df['date'] = pd.to_datetime(df['날짜'].str[:4] + df['날짜'].str[6:8] + df['날짜'].str[10:12])
    set_week(df, 'date')  # 날짜 데이터 정리
    df['oil'] = df[['year', 'week', '종가']].groupby(['year', 'week']).transform('mean')
    df.drop(columns = ['날짜', '종가'], inplace=True, axis=1)
    df.drop_duplicates(inplace=True)
    return df

In [359]:
df_oil = preprocess_oil(df_oil)
df_oil_dubai = preprocess_oil(df_oil_dubai)
df_oil_brent = preprocess_oil(df_oil_brent)

## Check

In [360]:
for i in range(1, 53):
    if i not in (list(df_oil_brent[df_oil_brent['year'] == 2019].sort_values(by=['year', 'week'])['week'])):
        print(i)

In [361]:
check_week(df_oil)
check_week(df_oil_dubai)
check_week(df_oil_brent)

missing 0 values
missing 0 values
missing 0 values


모든 데이터가 결측치는 없지만,  
df_oil 에 해당하는 wti 종가가 가장 예민하게 반응하는 값이므로  
이를 사용하도록 한다

## Add to Training Data

In [362]:
df_train = pd.merge(df_train, df_oil, how='left', on=['year', 'week'])

---

# Korea Weather

## Import Data

In [363]:
df_weather_kr = pd.read_csv(os.path.join(root, 'raw_weather_korea.csv'),
                            encoding='euc-kr',
                            usecols=["일시", "평균 풍속(m/s)", "평균 기온(°C)", "평균 수온(°C)"])

## Preprocess Date

In [364]:
df_weather_kr['date'] = pd.to_datetime(df_weather_kr['일시'].str[:4] + df_weather_kr['일시'].str[5:7] + df_weather_kr['일시'].str[8:])
set_week(df_weather_kr, 'date')

In [365]:
df_weather_kr.describe()

,평균 풍속(m/s),평균 기온(°C),평균 수온(°C),year,week
count,35181.000000,35107.000000,35300.000000,35558.000000,35558.000000
mean,5.732987,14.845689,16.884377,2018.369143,25.531554
std,2.705727,7.573504,5.945493,1.627939,15.146644
min,0.000000,-11.100000,1.100000,2015.000000,1.000000
25%,3.600000,8.800000,13.000000,2017.000000,13.000000
50%,5.400000,15.100000,16.600000,2018.000000,24.000000
75%,7.500000,21.000000,21.400000,2020.000000,39.000000
max,19.600000,34.600000,31.500000,2021.000000,53.000000


## Group
일자 별로 평균 구하기

In [366]:
df_weather_kr['wind_kr'] = df_weather_kr[['year', 'week', '평균 풍속(m/s)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['temperature_kr'] = df_weather_kr[['year', 'week', '평균 기온(°C)']].groupby(['year', 'week']).transform('mean')
df_weather_kr['water_temp_kr'] = df_weather_kr[['year', 'week', '평균 수온(°C)']].groupby(['year', 'week']).transform('mean')

## Drop Column

In [367]:
drop = ['일시', '평균 풍속(m/s)', '평균 기온(°C)', '평균 수온(°C)']

In [368]:
df_weather_kr.drop(columns=drop, inplace=True, axis=1)
df_weather_kr.drop_duplicates(inplace=True)

## Check

In [369]:
check_week(df_weather_kr)

missing 0 values


In [370]:
df_weather_kr.describe()

,year,week,wind_kr,temperature_kr,water_temp_kr
count,288.000000,288.000000,288.000000,288.000000,288.000000
mean,2018.267361,25.510417,5.723881,14.920397,16.953522
std,1.612682,15.112121,1.340964,7.177077,5.321755
min,2015.000000,1.000000,2.706723,-0.529915,8.389899
25%,2017.000000,12.750000,4.763866,8.648878,12.088782
50%,2018.000000,24.500000,5.668908,14.792355,16.279115
75%,2020.000000,38.250000,6.656410,21.312062,21.471801
max,2021.000000,53.000000,9.819643,28.610924,28.495798


## Add to Training Data

In [371]:
df_train = pd.merge(df_train, df_weather_kr, how='left', on=['year', 'week'])

---

# CPI

## Import Data

In [372]:
df_cpi_total = pd.read_csv(os.path.join(root, 'raw_cpi_total.csv'), encoding='cp949')  # 총 소비자물가지수
df_cpi_fish =  pd.read_csv(os.path.join(root, 'raw_cpi_fish.csv'), encoding='cp949')  # 수산물 소비자물가지수

## Drop Columns/Rows

In [373]:
df_cpi_total = df_cpi_total[df_cpi_total['지출목적별'] == "0 총지수"].drop(columns=['시도별', '지출목적별'], axis=1)
df_cpi_fish = df_cpi_fish[df_cpi_fish['지출목적별'] == "어류 및 수산"].drop(columns=['Unnamed: 0', '지출목적별'], axis=1)

## Pivoting

In [374]:
df_cpi_total = df_cpi_total.transpose().reset_index()
df_cpi_fish = df_cpi_fish.transpose().reset_index()

## Rename

In [375]:
df_cpi_total.rename(columns={0: 'cpi_total'}, inplace=True)
df_cpi_fish.rename(columns={0: 'cpi_fish'}, inplace=True)
print(df_cpi_total.columns, df_cpi_fish.columns)

Index(['index', 'cpi_total'], dtype='object') Index(['index', 'cpi_fish'], dtype='object')


## Preprocess Date

In [376]:
df_cpi_total['year'] = df_cpi_total['index'].str[:4].astype('int')
df_cpi_total['month'] = df_cpi_total['index'].str[6:8].astype('int')
df_cpi_fish['year'] = df_cpi_fish['index'].str[:4].astype('int')
df_cpi_fish['month'] = df_cpi_fish['index'].str[6:8].astype('int')

In [377]:
df_cpi_total.drop(columns=['index'], inplace=True)
df_cpi_fish.drop(columns=['index'], inplace=True)

In [378]:
# Merge df
df_cpi = pd.merge(left=df_cpi_total, right=df_cpi_fish, how='outer', on=['year', 'month'])

In [379]:
# create days by duplicating data
new_df = pd.DataFrame(columns=list(df_cpi.columns) + ['day'])
for i, row in df_cpi.iterrows():
    data = df_cpi.iloc[i]
    year, month, cpi_total, cpi_fish = data['year'].astype('int'), data['month'].astype('int'), data['cpi_total'], data['cpi_fish']
    for day in range(1, monthrange(year, month)[1]+1):
        new_df = new_df.append({'year': str(year),
                                'month': str(month),
                                'day': str(day),
                                'cpi_total': cpi_total,
                                'cpi_fish': cpi_fish},
                               ignore_index=True)
df_cpi = new_df

In [380]:
df_cpi['date'] = df_cpi['year'] + " " + df_cpi['month'] + " " + df_cpi['day']

In [381]:
set_week(df_cpi, 'date')

## 정리
- drop unused columns
- drop duplicates

In [382]:
df_cpi.drop(columns=['month', 'day'], inplace=True)
df_cpi.drop_duplicates(inplace=True)

## Check

In [383]:
check_week(df_cpi)

missing 0 values


In [384]:
df_cpi.describe()

,cpi_total,year,cpi_fish,week
count,348.000000,353.000000,353.000000,353.000000
mean,103.977759,2018.254958,112.068300,26.067989
std,1.901079,1.659215,6.449167,15.191427
min,100.220000,2015.000000,100.760000,1.000000
25%,102.720000,2017.000000,107.370000,13.000000
50%,104.350000,2018.000000,111.960000,26.000000
75%,105.460000,2020.000000,118.440000,39.000000
max,107.460000,2021.000000,122.010000,53.000000


In [385]:
df_cpi.sample(5)

,cpi_total,year,cpi_fish,week
230,100.60,2016,102.62,29
1315,104.56,2019,113.72,28
1693,104.86,2020,119.22,30
251,100.86,2016,101.69,32
1091,104.71,2018,112.51,48


## Add to Training Data

In [386]:
df_train = pd.merge(df_train, df_cpi, how='left', on=['year', 'week'])

---

# Exchange Rate

In [387]:
exchange_list = [pd.read_csv(os.path.join(root, 'raw_exchange_chile.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_china.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_norway.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_peru.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_thai.csv'), usecols=["날짜", "종가"]),
                 pd.read_csv(os.path.join(root, 'raw_exchange_vietnam.csv'), usecols=["날짜", "종가"])]

In [388]:
ctry_name = ['칠레', '중국', '노르웨이', '페루', '태국', '베트남']
for i in range(len(exchange_list)):
    set_week(exchange_list[i], '날짜')
    check_week(exchange_list[i])
    exchange_list[i]['CTRY_2'] = ctry_name[i]

missing 0 values
missing 0 values
missing 0 values
missing 0 values
missing 0 values
missing 0 values


In [389]:
df_exchange = pd.concat(exchange_list)

## Rename

In [390]:
df_exchange.rename(columns={"종가": 'exchange'}, inplace=True)

## Add to Training Data

In [391]:
df_train = pd.merge(df_train, df_exchange, how='left', on=['year', 'week', 'CTRY_2'])

In [392]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,절단,한쪽껍질붙은,외투막,난포선,지느러미,자숙,활,턱살,줄기,알,내장,횟감,껍질,집게다리,살,냉동,목살,머리살,개아지살,훈제,머리_외화획득용,포장횟감,볼살,염장,곤이,슬라이스(S),다리,간,꼬리_외화획득용,캐비아대용,냉장,머리,필렛(F),동체,창난,눈살,건조,year,week,name_cnt,is_processed,import_cnt,rain,wind,temperature,oil,wind_kr,temperature_kr,water_temp_kr,cpi_total,cpi_fish,exchange
67873,수산물,중국,중국,판매용,패류 멍게류,조개 백합 대합,비너스백합,활,0.988259,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,44,164,False,1,3.047805,2.760976,13.616098,55.358,5.109091,17.680165,19.944628,104.87,114.92,0.141500
43744,수산물,베네수엘라,베네수엘라,판매용,어류,갈치,갈치,냉동,3.500000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,24,952,False,1,NaN,NaN,NaN,66.210,5.374790,18.942857,19.451261,104.13,113.07,NaN
30625,수산물,기니,기니,판매용,어류,조기 보구치 강다리,긴가이석태,냉동,4.809640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2017,39,408,False,1,NaN,NaN,NaN,51.894,5.947059,22.193277,23.118487,103.49,106.19,NaN
26516,수산물,파키스탄,파키스탄,판매용,갑각류,게,게,냉동,2.395111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2017,27,418,False,1,NaN,NaN,NaN,45.816,4.500000,23.886957,22.460504,102.78,106.70,NaN
42554,수산물,베트남,베트남,판매용,연체류 해물모듬,갑오징어,갑오징어,"냉동,다리",4.800000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2018,21,783,False,2,8.786667,1.600000,28.208889,70.360,4.109649,16.643363,16.298230,104.34,113.01,0.000044


# Final

## One-hot Encoding

In [393]:
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']

In [394]:
df_train = pd.get_dummies(df_train, columns=one_hot)

## Drop Columns

In [397]:
drop = ['P_TYPE', 'P_IMPORT_TYPE']

In [398]:
df_train.drop(columns = drop, inplace=True)

In [399]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,P_PRICE,절단,한쪽껍질붙은,외투막,난포선,지느러미,자숙,활,턱살,줄기,알,내장,횟감,껍질,집게다리,살,냉동,목살,머리살,개아지살,훈제,머리_외화획득용,포장횟감,볼살,염장,곤이,슬라이스(S),다리,간,꼬리_외화획득용,캐비아대용,냉장,머리,필렛(F),동체,창난,눈살,건조,year,week,name_cnt,is_processed,import_cnt,rain,wind,temperature,oil,wind_kr,temperature_kr,water_temp_kr,cpi_total,cpi_fish,exchange,CTRY_1_가나,CTRY_1_감비아,CTRY_1_그리스,CTRY_1_그린란드,CTRY_1_기니,CTRY_1_기니비사우,CTRY_1_나미비아,CTRY_1_남아프리카 공화국,CTRY_1_네덜란드,CTRY_1_노르웨이,CTRY_1_뉴질랜드,CTRY_1_니카라과,CTRY_1_대만,CTRY_1_대한민국,CTRY_1_라이베리아,CTRY_1_라트비아,CTRY_1_러시아,CTRY_1_루마니아,CTRY_1_리비아,CTRY_1_마다가스카르,CTRY_1_말레이시아,CTRY_1_멕시코,CTRY_1_모로코,CTRY_1_모리타니,CTRY_1_모잠비크,CTRY_1_몰타,CTRY_1_미국,CTRY_1_미얀마,CTRY_1_미크로네시아 연방,CTRY_1_바누아투,CTRY_1_바레인,CTRY_1_방글라데시,CTRY_1_베네수엘라,CTRY_1_베트남,CTRY_1_불가리아,CTRY_1_브라질,CTRY_1_사우디아라비아,CTRY_1_사이프러스,CTRY_1_세네갈,CTRY_1_세이셸,CTRY_1_세인트빈센트 그레나딘,CTRY_1_소말리아,CTRY_1_수리남,CTRY_1_스리랑카,CTRY_1_스페인,CTRY_1_시에라리온,CTRY_1_싱가포르,CTRY_1_아랍에미리트,CTRY_1_아르헨티나,CTRY_1_아이슬란드,CTRY_1_아일랜드,CTRY_1_알제리,CTRY_1_앙골라,CTRY_1_에스토니아,CTRY_1_에콰도르,CTRY_1_영국,CTRY_1_오만,CTRY_1_우루과이,CTRY_1_우크라이나,CTRY_1_이란,CTRY_1_이집트,CTRY_1_이탈리아,CTRY_1_인도,CTRY_1_인도네시아,CTRY_1_일본,CTRY_1_중국,CTRY_1_칠레,CTRY_1_캐나다,CTRY_1_콜롬비아,CTRY_1_쿠바,CTRY_1_쿡 제도,CTRY_1_크로아티아,CTRY_1_키리바시,CTRY_1_태국,CTRY_1_터키,CTRY_1_튀니지,CTRY_1_파나마,CTRY_1_파키스탄,CTRY_1_파푸아뉴기니,CTRY_1_팔라우,CTRY_1_페루,CTRY_1_포르투갈,CTRY_1_포클랜드 제도,CTRY_1_프랑스,CTRY_1_피지,CTRY_1_필리핀,CTRY_1_호주,CTRY_2_가나,CTRY_2_감비아,CTRY_2_그리스,CTRY_2_기니,CTRY_2_기니비사우,CTRY_2_기타(ZZ),CTRY_2_나미비아,CTRY_2_남아프리카 공화국,CTRY_2_네덜란드,CTRY_2_노르웨이,CTRY_2_뉴질랜드,CTRY_2_대만,CTRY_2_덴마크,CTRY_2_독일,CTRY_2_라이베리아,CTRY_2_러시아,CTRY_2_루마니아,CTRY_2_마다가스카르,CTRY_2_말레이시아,CTRY_2_멕시코,CTRY_2_모로코,CTRY_2_모리셔스,CTRY_2_모리타니,CTRY_2_모잠비크,CTRY_2_몰타,CTRY_2_미국,CTRY_2_미얀마,CTRY_2_미크로네시아 연방,CTRY_2_바누아투,CTRY_2_바레인,CTRY_2_방글라데시,CTRY_2_베네수엘라,CTRY_2_베트남,CTRY_2_벨리즈,CTRY_2_북한,CTRY_2_불가리아,CTRY_2_브라질,CTRY_2_사모아,CTRY_2_사우디아라비아,CTRY_2_세네갈,CTRY_2_세이셸,CTRY_2_솔로몬 제도,CTRY_2_수리남,CTRY_2_스리랑카,CTRY_2_스웨덴,CTRY_2_스페인,CTRY_2_시에라리온,CTRY_2_싱가포르,CTRY_2_아랍에미리트,CTRY_2_아르헨티나,CTRY_2_아이슬란드,CTRY_2_아일랜드,CTRY_2_앙골라,CTRY_2_에콰도르,CTRY_2_영국,CTRY_2_오만,CTRY_2_우루과이,CTRY_2_우크라이나,CTRY_2_이란,CTRY_2_이집트,CTRY_2_이탈리아,CTRY_2_인도,CTRY_2_인도네시아,CTRY_2_일본,CTRY_2_중국,CTRY_2_지부티,CTRY_2_칠레,CTRY_2_캐나다,CTRY_2_콜롬비아,CTRY_2_쿠바,CTRY_2_크로아티아,CTRY_2_키리바시,CTRY_2_태국,CTRY_2_터키,CTRY_2_투발루,CTRY_2_튀니지,CTRY_2_파나마,CTRY_2_파키스탄,CTRY_2_파푸아뉴기니,CTRY_2_팔라우,CTRY_2_페루,CTRY_2_포르투갈,CTRY_2_포클랜드 제도,CTRY_2_프랑스,CTRY_2_피지,CTRY_2_필리핀,CTRY_2_호주,CTRY_2_홍콩,P_PURPOSE_반송품(기타),P_PURPOSE_외화획득용 원료,P_PURPOSE_자사제품제조용,P_PURPOSE_판매용,P_PURPOSE_합작,CATEGORY_1_갑각류,CATEGORY_1_기타 수입식품,CATEGORY_1_알 곤이류,CATEGORY_1_어류,CATEGORY_1_연체류 해물모듬,CATEGORY_1_젓갈류 해조류 해파리,CATEGORY_1_패류 멍게류,CATEGORY_2_가리비,CATEGORY_2_가물치,CATEGORY_2_가사리,CATEGORY_2_가오리,CATEGORY_2_가자미,CATEGORY_2_가재 랍스타,CATEGORY_2_갈치,CATEGORY_2_갑오징어,CATEGORY_2_개복치,CATEGORY_2_개불,CATEGORY_2_게,CATEGORY_2_고동,CATEGORY_2_고등어,CATEGORY_2_고시래기,CATEGORY_2_골뱅이,CATEGORY_2_광어 넙치,CATEGORY_2_김,CATEGORY_2_꼬막,CATEGORY_2_꼴뚜기,CATEGORY_2_꽁치 학꽁치,CATEGORY_2_꽁치 확꽁치,CATEGORY_2_낙지,CATEGORY_2_날치알,CATEGORY_2_남극빙어,CATEGORY_2_노래미,CATEGORY_2_농어,CATEGORY_2_능성어 붉바리 바리,CATEGORY_2_다시마,CATEGORY_2_달고기,CATEGORY_2_대구,CATEGORY_2_대구알,CATEGORY_2_도다리,CATEGORY_2_도미 감성돔 돔류,CATEGORY_2_망둑어,CATEGORY_2_멍게,CATEGORY_2_메기 동자개,CATEGORY_2_메로,CATEGORY_2_멸치,CATEGORY_2_명란(명태알),CATEGORY_2_명태,CATEGORY_2_문어,CATEGORY_2_물메기(곰치),CATEGORY_2_미꾸라지,CATEGORY_2_미역,CATEGORY_2_민물붕어,CATEGORY_2_민어 점성어,CATEGORY_2_밀크피시,CATEGORY_2_바지락,CATEGORY_2_방어,CATEGORY_2_밴댕이,CATEGORY_2_버터플라이 킹피쉬,CATEGORY_2_벤자리 알롱이,CATEGORY_2_병어,CATEGORY_2_보리멸,CATEGORY_2_복어,CATEGORY_2_부세,CATEGORY_2_붉평치(만다이 꽃돔),CATEGORY_2_삼치,CATEGORY_2_상어 고래,CATEGORY_2_새우,CATEGORY_2_샛돔류알,CATEGORY_2_서대 박대 페루다,CATEGORY_2_성게알,CATEGORY_2_소라,CATEGORY_2_송어,CATEGORY_2_쏘가리,CATEGORY_2_아귀,CATEGORY_2_양미리 정어리,CATEGORY_2_양태,CATEGORY_2_어류 기타,CATEGORY_2_연어,CATEGORY_2_연어알,CATEGORY_2_열빙어(시샤모),CATEGORY_2_열빙어(시샤모)알,CATEGORY_2_오징어,CATEGORY_2_옥돔,CATEGORY_2_우럭 볼락,CATEGORY_2_우렁 다슬기,CATEGORY_2_은민대구알,CATEGORY_2_임연수,CATEGORY_2_잉어,CATEGORY_2_자라,CATEGORY_2_장어,CATEGORY_2_재첩,CATEGORY_2_적어 눈볼대,CATEGORY_2_전갱기 매가리,CATEGORY_2_전갱이 매가리,CATEGORY_2_전복,CATEGORY_2_전어,CATEGORY_2_조개,CATEGORY_2_조개 백합 대합,CATEGORY_2_조기

## Save Files

In [ ]:
def save_file(df, file_name):
    df.to_csv(os.path.join(root, file_name), encoding='utf-8', index=False)

In [ ]:
save_file(df_oil, 'preprocessed_oil.csv')  # 원유 종가
save_file(df_weather_kr, 'preprocessed_weather_korea.csv')  # 한국 날씨
save_file(df_cpi, 'preprocessed_cpi.csv')  # 소비자물가지수
save_file(df_exchange, 'preprocessed_exchange.csv')  # 환율
save_file(df_train, 'preprocessed_train.csv')  # 최종 df